In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import math

In [2]:
mask = np.zeros((144,4)).astype(int)
#mask

In [3]:
topIndices = np.arange(144).reshape((12,12))
topIndices

array([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11],
       [ 12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23],
       [ 24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35],
       [ 36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47],
       [ 48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59],
       [ 60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71],
       [ 72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83],
       [ 84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95],
       [ 96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107],
       [108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119],
       [120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131],
       [132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143]])

In [4]:
rolledTopIndices = np.roll(topIndices, 3, axis=(0,1))
rolledTopIndices

array([[117, 118, 119, 108, 109, 110, 111, 112, 113, 114, 115, 116],
       [129, 130, 131, 120, 121, 122, 123, 124, 125, 126, 127, 128],
       [141, 142, 143, 132, 133, 134, 135, 136, 137, 138, 139, 140],
       [  9,  10,  11,   0,   1,   2,   3,   4,   5,   6,   7,   8],
       [ 21,  22,  23,  12,  13,  14,  15,  16,  17,  18,  19,  20],
       [ 33,  34,  35,  24,  25,  26,  27,  28,  29,  30,  31,  32],
       [ 45,  46,  47,  36,  37,  38,  39,  40,  41,  42,  43,  44],
       [ 57,  58,  59,  48,  49,  50,  51,  52,  53,  54,  55,  56],
       [ 69,  70,  71,  60,  61,  62,  63,  64,  65,  66,  67,  68],
       [ 81,  82,  83,  72,  73,  74,  75,  76,  77,  78,  79,  80],
       [ 93,  94,  95,  84,  85,  86,  87,  88,  89,  90,  91,  92],
       [105, 106, 107,  96,  97,  98,  99, 100, 101, 102, 103, 104]])

In [5]:
ma = mask
ma[rolledTopIndices[:6,:6],0] = 1
ma[rolledTopIndices[:6,6:12],1] = 1
ma[rolledTopIndices[6:12,:6],2] = 1
ma[rolledTopIndices[6:12,6:12],3] = 1
ma

array([[1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0,

In [6]:
ma = mask
ma[topIndices[:6,:6],0] = 1
ma[topIndices[:6,6:12],1] = 1
ma[topIndices[6:12,:6],2] = 1
ma[topIndices[6:12,6:12],3] = 1
ma

array([[1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 1, 0, 0],
       [1, 1, 0, 0],
       [1, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [1, 1, 0, 0],
       [1, 1, 0, 0],
       [1, 1, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 1, 0, 0],
       [1, 1, 0, 0],
       [1, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [1, 1, 0, 0],
       [1, 1, 0, 0],
       [1, 1, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 1, 0, 0],
       [1, 1, 0, 0],
       [1, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [1, 1, 0, 0],
       [1, 1, 0, 0],
       [1, 1, 0, 0],
       [1, 0, 1, 0],
       [1, 0, 1, 0],
       [1, 0, 1, 0],
       [1, 0, 0, 1],
       [1, 0, 0, 1],
       [1, 0, 0, 1],
       [0, 1, 0, 1],
       [0, 1, 0, 1],
       [0, 1, 0, 1],
       [0, 1, 1, 0],
       [0, 1, 1, 0],
       [0, 1,